In [1]:
import os

In [21]:
%pwd

'c:\\Users\\kosht\\Documents\\Learning\\Youtube\\End To End NLP Project Implementation With Deployment Github Action- Text Summarization- Krish Naik\\code\\Text-Summarizer-Project'

In [3]:
os.chdir("../")

In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
  root_dir: Path
  data_path: Path
  tokenizer_name: Path

In [23]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [27]:

class ConfigurationManger:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifact_root])

    def get_data_transformation_config(self)->DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [30]:
import os
from urllib import request
from textSummarizer.utils.common import get_size
from textSummarizer.logging import logger
import zipfile
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [58]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig) -> None:
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch["dialogue"], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            "input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings["input_ids"]
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))
    


In [59]:
try:
    config = ConfigurationManger()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2023-12-30 16:41:52,489 INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-30 16:41:52,497 INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-30 16:41:52,502 INFO: common: created directory at: artifacts]
[2023-12-30 16:41:52,506 INFO: common: created directory at: artifacts/data_transformation]


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 71091.37 examples/s]
